In [ ]:
import pandas as pd
import json
import os
import ast
import re
import numpy as np
import matplotlib.pyplot as plt

from transformers import (

    set_seed,
    
)

import torch
from time import time
import faiss
from sentence_transformers import SentenceTransformer
# For wandb
import wandb
# Set seed
import pickle
set_seed(42)

import seaborn as sns
from copy import deepcopy
from tqdm import tqdm


In [11]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis


In [ ]:
languages = ['deu', 'eng', 'vie']

In [ ]:
language = 'deu'
owilix_df = pd.read_csv("data/raw/m1_" + f"{language}_raw_owilix.csv")
owilix_df = owilix_df.drop(columns="Unnamed: 0")

In [9]:
df_human = pd.read_csv("data/human/1_combine_hate_ds.csv")
df_human['len_text'] = df_human['text'].str.len()
df_human = df_human[df_human['len_text'] <= 300]
df_human = df_human[df_human['language'] == language]

print(df_human.shape)

(15588, 7)


In [ ]:
df_human_text = df_human['text'].tolist()

In [ ]:
res = faiss.StandardGpuResources() 

In [ ]:
human_cmts = df_human_text
batch_size = 100000
k = 2
model = SentenceTransformer('all-MiniLM-L6-v2')

hate_speech_embeddings = model.encode(human_cmts, convert_to_numpy=True)
dimension = hate_speech_embeddings.shape[1]
 
delta_s = hate_speech_embeddings
delta_k = human_cmts
filter_cmts =[]

for index_b in range(0, int(len(owilix_df)/batch_size) + 1): 
    print(index_b, len(filter_cmts))
    filter_cmts_index = []
    owilix_small_1 =  owilix_df.iloc[index_b * batch_size: (index_b+1)*batch_size] 
    index = faiss.IndexFlatL2(dimension)
    gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index)
    cmts = owilix_small_1['text'].tolist()
    comment_embeddings = model.encode(cmts, convert_to_numpy=True)
    gpu_index_flat.add(comment_embeddings)
    for i, sample_embedding in enumerate(hate_speech_embeddings):
        distances, indices = gpu_index_flat.search(np.array([sample_embedding]), k)  # Perform the search
        for j in range(k):
            filter_cmts_index.append(int(indices[0][j]))
    filter_cmts_index = np.unique(filter_cmts_index)
    for j in filter_cmts_index:
        filter_cmts.append(cmts[j])

In [ ]:
import json
with open("data/raw" + f"m1_{language}_filtered"  + ".json", 'w') as file:
    json.dump(filter_cmts, file)